In [ ]:
pip install opencv-python mediapipe numpy matplotlib


In [ ]:
import cv2
import numpy as np
import mediapipe as mp
import matplotlib.pyplot as plt

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)

# Initialize MediaPipe Holistics
mp_holistic = mp.solutions.holistic
holistic_model = mp_holistic.Holistic(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

# Load Haar Cascade for face detection
face_haar_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Initialize lists to store data for graphs
left_hand = []
right_hand = []

# Set the graph display parameters
graph_height = 200
line_width = 1

# Set video dimensions
output_width = 800
output_height = 600

# Create a video capture object
cap = cv2.VideoCapture(0)

# Create an output video writer
output_video = 'output_video.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video, fourcc, 20, (output_width, output_height))

# Initialize frame index
frame_index = 0

# Process each frame from the camera feed
while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame_grey = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_haar_cascade.detectMultiScale(frame_grey)

    # Detect pose in the frame
    results = pose.process(frame_rgb)

    frame_rgb.flags.writeable = False
    hand_results = holistic_model.process(frame_rgb)
    frame_rgb.flags.writeable = True

    left_hand_sum_distance = 0
    if hand_results.left_hand_landmarks is not None:
        mp.solutions.drawing_utils.draw_landmarks(frame, hand_results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
        left_hand_landmarks = hand_results.left_hand_landmarks.landmark
        for landmark in left_hand_landmarks:
            x = landmark.x
            y = landmark.y
            z = landmark.z
            distance = np.sqrt(x**2 + y**2 + z**2)
            left_hand_sum_distance += distance
        left_hand.append(left_hand_sum_distance)
    else:
        left_hand.append(None)

    right_hand_sum_distance = 0
    if hand_results.right_hand_landmarks is not None:
        mp.solutions.drawing_utils.draw_landmarks(frame, hand_results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
        right_hand_landmarks = hand_results.right_hand_landmarks.landmark
        for landmark in right_hand_landmarks:
            x = landmark.x
            y = landmark.y
            z = landmark.z
            distance = np.sqrt(x**2 + y**2 + z**2)
            right_hand_sum_distance += distance
        right_hand.append(right_hand_sum_distance)
    else:
        right_hand.append(None)

    # Update the frame index
    frame_index += 1

    # Filter out None values and calculate the min and max values for each graph
    filtered_left_hand = [point for point in left_hand if point is not None]
    filtered_right_hand = [point for point in right_hand if point is not None]
    
    min_left_hand = min(filtered_left_hand) if filtered_left_hand else 0
    max_left_hand = max(filtered_left_hand) if filtered_left_hand else 0
    min_right_hand = min(filtered_right_hand) if filtered_right_hand else 0
    max_right_hand = max(filtered_right_hand) if filtered_right_hand else 0

    # Calculate engagement level
    engagement_level = [(lh + rh) / 2 if lh is not None and rh is not None else None for lh, rh in zip(left_hand, right_hand)]
    filtered_engagement_level = [e for e in engagement_level if e is not None]
    
    min_engagement = min(filtered_engagement_level) if filtered_engagement_level else 0
    max_engagement = max(filtered_engagement_level) if filtered_engagement_level else 1

    # Create separate figures for each graph
    plt.figure(figsize=(output_width / 100, 3 * graph_height / 100))  # Adjust graph size

    # Hand Movement Graph
    plt.subplot(3, 1, 1)
    plt.plot(np.arange(frame_index), left_hand, color='red', linewidth=line_width, marker='o', markersize=1, label='Left hand movement')
    plt.plot(np.arange(frame_index), right_hand, color='blue', linewidth=line_width, marker='o', markersize=1, label='Right hand movement')
    plt.ylim([min(min_left_hand, min_right_hand), max(max_left_hand, max_right_hand)])
    plt.xlim([0, frame_index])
    plt.axis('on')
    plt.ylabel('Hand Movements', fontsize=12)
    plt.legend()

    # Engagement Graph
    plt.subplot(3, 1, 2)
    plt.plot(np.arange(frame_index), engagement_level, color='green', linewidth=line_width, marker='o', markersize=1, label='Engagement Level')
    plt.ylim([min_engagement, max_engagement])
    plt.xlim([0, frame_index])
    plt.axis('on')
    plt.ylabel('Engagement Level', fontsize=12)
    plt.legend()

    plt.tight_layout()

    # Save the graph with labels
    plt.savefig('temp_graph.png', bbox_inches='tight', pad_inches=0, transparent=True)
    plt.clf()

    graph = cv2.imread('temp_graph.png')

    # Resize the graph to match the video width and increased height
    graph = cv2.resize(graph, (output_width, 3 * graph_height))

    # Display the input video
    cv2.imshow('Input Video', frame)

    # Display the graph with labels
    cv2.imshow('Graph', graph)

    # Write the frame with graph to the output video
    out.write(cv2.resize(frame, (output_width, output_height)))

    # Break the loop if the 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release video objects and close OpenCV windows
cap.release()
out.release()
cv2.destroyAllWindows()


In [ ]:
import cv2
import numpy as np
import mediapipe as mp
import matplotlib.pyplot as plt
from concurrent.futures import ThreadPoolExecutor
import time

# Initialize MediaPipe Pose and Holistic
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)
mp_holistic = mp.solutions.holistic
holistic_model = mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

# Load Haar Cascade for face detection
face_haar_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Initialize lists to store data for graphs
left_hand = []
right_hand = []
shoulder_midpoints = []
head_turn_angles = []
engagement_level = []

# Set the graph display parameters
graph_height = 200
line_width = 1

# Set video dimensions
output_width = 800
output_height = 600

# Create a video capture object
cap = cv2.VideoCapture(0)

# Create an output video writer
output_video = 'output_video.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video, fourcc, 20, (output_width, output_height))

# Initialize frame index
frame_index = 0

def process_frame(frame, frame_index):
    global left_hand, right_hand, shoulder_midpoints, head_turn_angles, engagement_level

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame_grey = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_haar_cascade.detectMultiScale(frame_grey)

    # Detect pose in the frame
    results = pose.process(frame_rgb)
    frame_rgb.flags.writeable = False
    hand_results = holistic_model.process(frame_rgb)
    frame_rgb.flags.writeable = True

    # Draw landmarks and calculate distances for left and right hands
    left_hand_sum_distance = 0
    right_hand_sum_distance = 0

    if hand_results.left_hand_landmarks:
        mp_drawing.draw_landmarks(frame, hand_results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
        left_hand_landmarks = hand_results.left_hand_landmarks.landmark
        for landmark in left_hand_landmarks:
            x = landmark.x
            y = landmark.y
            z = landmark.z
            distance = np.sqrt(x**2 + y**2 + z**2)
            left_hand_sum_distance += distance
        left_hand.append(left_hand_sum_distance)
    else:
        left_hand.append(None)

    if hand_results.right_hand_landmarks:
        mp_drawing.draw_landmarks(frame, hand_results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
        right_hand_landmarks = hand_results.right_hand_landmarks.landmark
        for landmark in right_hand_landmarks:
            x = landmark.x
            y = landmark.y
            z = landmark.z
            distance = np.sqrt(x**2 + y**2 + z**2)
            right_hand_sum_distance += distance
        right_hand.append(right_hand_sum_distance)
    else:
        right_hand.append(None)

    # Draw pose landmarks
    if results.pose_landmarks:
        mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        left_shoulder = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER]
        right_shoulder = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER]
        shoulder_midpoint = (left_shoulder.x + right_shoulder.x) / 2
        shoulder_midpoints.append(shoulder_midpoint)

        left_eye = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_EYE]
        right_eye = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_EYE]
        nose = results.pose_landmarks.landmark[mp_pose.PoseLandmark.NOSE]

        # Calculate head turn angle
        if left_eye and right_eye and nose:
            eye_line_vector = np.array([right_eye.x - left_eye.x, right_eye.y - left_eye.y])
            nose_vector = np.array([nose.x - (left_eye.x + right_eye.x) / 2, nose.y - (left_eye.y + right_eye.y) / 2])
            cosine_angle = np.dot(eye_line_vector, nose_vector) / (np.linalg.norm(eye_line_vector) * np.linalg.norm(nose_vector))
            head_turn_angle = np.arccos(cosine_angle) * (180 / np.pi)
            head_turn_angles.append(head_turn_angle)
        else:
            head_turn_angles.append(None)
    else:
        shoulder_midpoints.append(None)
        head_turn_angles.append(None)

    # Calculate engagement level
    avg_hand_movement = (left_hand_sum_distance + right_hand_sum_distance) / 2 if left_hand_sum_distance and right_hand_sum_distance else None
    avg_head_turn = np.mean(head_turn_angles[-5:]) if head_turn_angles[-5:] else None
    if avg_hand_movement and avg_head_turn:
        engagement = avg_hand_movement + avg_head_turn
    else:
        engagement = None
    engagement_level.append(engagement)

    # Filter out None values for graphs
    filtered_left_hand = [point for point in left_hand if point is not None]
    filtered_right_hand = [point for point in right_hand if point is not None]
    filtered_shoulder_midpoints = [point for point in shoulder_midpoints if point is not None]
    filtered_head_turn_angles = [point for point in head_turn_angles if point is not None]
    filtered_engagement_level = [e for e in engagement_level if e is not None]

    # Calculate min and max values for y-axis limits
    min_left_hand = min(filtered_left_hand) if filtered_left_hand else 0
    max_left_hand = max(filtered_left_hand) if filtered_left_hand else 1
    min_right_hand = min(filtered_right_hand) if filtered_right_hand else 0
    max_right_hand = max(filtered_right_hand) if filtered_right_hand else 1
    min_shoulder_midpoint = min(filtered_shoulder_midpoints) if filtered_shoulder_midpoints else 0
    max_shoulder_midpoint = max(filtered_shoulder_midpoints) if filtered_shoulder_midpoints else 1
    min_head_turn_angle = min(filtered_head_turn_angles) if filtered_head_turn_angles else 0
    max_head_turn_angle = max(filtered_head_turn_angles) if filtered_head_turn_angles else 1
    min_engagement = min(filtered_engagement_level) if filtered_engagement_level else 0
    max_engagement = max(filtered_engagement_level) if filtered_engagement_level else 1

    # Create separate figures for each graph
    plt.figure(figsize=(output_width / 100, 4 * graph_height / 100))  # Adjust graph size

    # Hand Movement Graph
    plt.subplot(4, 1, 1)
    if len(filtered_left_hand) > 0 and len(filtered_right_hand) > 0:
        plt.plot(np.arange(len(filtered_left_hand)), filtered_left_hand, color='red', linewidth=line_width, marker='o', markersize=1, label='Left hand movement')
        plt.plot(np.arange(len(filtered_right_hand)), filtered_right_hand, color='blue', linewidth=line_width, marker='o', markersize=1, label='Right hand movement')
    plt.ylim([min(min_left_hand, min_right_hand), max(max_left_hand, max_right_hand)])
    plt.xlim([0, frame_index])
    plt.axis('on')
    plt.ylabel('Hand Movements', fontsize=12)
    plt.legend()

    # Shoulder Midpoints Graph
    plt.subplot(4, 1, 2)
    if len(filtered_shoulder_midpoints) > 0:
        plt.plot(np.arange(len(filtered_shoulder_midpoints)), filtered_shoulder_midpoints, color='orange', linewidth=line_width, marker='o', markersize=1, label='Shoulder Midpoints')
    plt.ylim([min_shoulder_midpoint, max_shoulder_midpoint])
    plt.xlim([0, frame_index])
    plt.axis('on')
    plt.ylabel('Shoulder Midpoints', fontsize=12)
    plt.legend()

    # Head Turn Angles Graph
    plt.subplot(4, 1, 3)
    if len(filtered_head_turn_angles) > 0:
        plt.plot(np.arange(len(filtered_head_turn_angles)), filtered_head_turn_angles, color='purple', linewidth=line_width, marker='o', markersize=1, label='Head Turn Angles')
    plt.ylim([min_head_turn_angle, max_head_turn_angle])
    plt.xlim([0, frame_index])
    plt.axis('on')
    plt.ylabel('Head Turn Angles', fontsize=12)
    plt.legend()

    # Engagement Level Graph
    plt.subplot(4, 1, 4)
    if len(filtered_engagement_level) > 0:
        plt.plot(np.arange(len(filtered_engagement_level)), filtered_engagement_level, color='green', linewidth=line_width, marker='o', markersize=1, label='Engagement Level')
    plt.ylim([min_engagement, max_engagement])
    plt.xlim([0, frame_index])
    plt.axis('on')
    plt.ylabel('Engagement Level', fontsize=12)
    plt.legend()

    plt.tight_layout()

    # Save the graph with labels
    plt.savefig('temp_graph.png', bbox_inches='tight', pad_inches=0, transparent=True)
    plt.clf()

    graph = cv2.imread('temp_graph.png')
    graph = cv2.resize(graph, (output_width, 4 * graph_height))

    return frame, graph

def main():
    global frame_index

    with ThreadPoolExecutor(max_workers=2) as executor:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            future = executor.submit(process_frame, frame, frame_index)
            processed_frame, graph = future.result()

            # Update the frame index
            frame_index += 1

            # Display the input video
            cv2.imshow('Input Video', processed_frame)

            # Display the graph with labels
            cv2.imshow('Graph', graph)

            # Write the frame with graph to the output video
            out.write(cv2.resize(processed_frame, (output_width, output_height)))

            # Break the loop if the 'q' key is pressed
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

            # Slow down the frame rate
            time.sleep(0.1)

    # Release video objects and close OpenCV windows
    cap.release()
    out.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


In [ ]:
# upper codes are fine

In [ ]:
import cv2
import numpy as np
import mediapipe as mp
import matplotlib.pyplot as plt
from concurrent.futures import ThreadPoolExecutor
import time

# Initialize MediaPipe Pose and Holistic
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)
mp_holistic = mp.solutions.holistic
holistic_model = mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

# Load Haar Cascade for face detection
face_haar_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Initialize lists to store data for graphs
left_hand = []
right_hand = []
shoulder_midpoints = []
head_turn_angles = []
engagement_level = []

# Set the graph display parameters
graph_height = 200
line_width = 1

# Set video dimensions
output_width = 800
output_height = 600

# Create a video capture object
cap = cv2.VideoCapture(0)

# Create an output video writer
output_video = 'output_video.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video, fourcc, 20, (output_width, output_height))

# Initialize frame index
frame_index = 0

def calculate_engagement(left_hand, right_hand, shoulder_midpoints, head_turn_angles):
    # Use recent history to determine engagement level
    history_len = 10  # Number of frames to consider for engagement calculation

    # Slicing the recent history
    recent_left_hand = left_hand[-history_len:]
    recent_right_hand = right_hand[-history_len:]
    recent_shoulder_midpoints = shoulder_midpoints[-history_len:]
    recent_head_turn_angles = head_turn_angles[-history_len:]

    # Calculate average movements
    avg_left_hand = np.nanmean([x for x in recent_left_hand if x is not None])
    avg_right_hand = np.nanmean([x for x in recent_right_hand if x is not None])
    avg_shoulder = np.nanmean([x for x in recent_shoulder_midpoints if x is not None])
    avg_head_turn = np.nanmean([x for x in recent_head_turn_angles if x is not None])

    # Engagement calculation based on movement thresholds
    hand_threshold = 0.1  # Movement threshold for hands
    shoulder_threshold = 0.05  # Movement threshold for shoulders
    head_turn_threshold = 5.0  # Angle threshold for head turns (degrees)

    hand_movement = (avg_left_hand is not None and avg_left_hand > hand_threshold) or (avg_right_hand is not None and avg_right_hand > hand_threshold)
    shoulder_movement = avg_shoulder is not None and avg_shoulder > shoulder_threshold
    head_movement = avg_head_turn is not None and avg_head_turn > head_turn_threshold

    if hand_movement and shoulder_movement and head_movement:
        return 'inspired'
    elif hand_movement and (shoulder_movement or head_movement):
        return 'interactive'
    else:
        return 'attentive'

def process_frame(frame, frame_index):
    global left_hand, right_hand, shoulder_midpoints, head_turn_angles, engagement_level

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame_grey = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_haar_cascade.detectMultiScale(frame_grey)

    # Detect pose in the frame
    results = pose.process(frame_rgb)
    frame_rgb.flags.writeable = False
    hand_results = holistic_model.process(frame_rgb)
    frame_rgb.flags.writeable = True

    # Draw landmarks and calculate distances for left and right hands
    left_hand_sum_distance = 0
    right_hand_sum_distance = 0

    if hand_results.left_hand_landmarks:
        mp_drawing.draw_landmarks(frame, hand_results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
        left_hand_landmarks = hand_results.left_hand_landmarks.landmark
        for landmark in left_hand_landmarks:
            x = landmark.x
            y = landmark.y
            z = landmark.z
            distance = np.sqrt(x**2 + y**2 + z**2)
            left_hand_sum_distance += distance
        left_hand.append(left_hand_sum_distance)
    else:
        left_hand.append(None)

    if hand_results.right_hand_landmarks:
        mp_drawing.draw_landmarks(frame, hand_results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
        right_hand_landmarks = hand_results.right_hand_landmarks.landmark
        for landmark in right_hand_landmarks:
            x = landmark.x
            y = landmark.y
            z = landmark.z
            distance = np.sqrt(x**2 + y**2 + z**2)
            right_hand_sum_distance += distance
        right_hand.append(right_hand_sum_distance)
    else:
        right_hand.append(None)

    # Draw pose landmarks
    if results.pose_landmarks:
        mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        left_shoulder = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER]
        right_shoulder = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER]
        shoulder_midpoint = (left_shoulder.x + right_shoulder.x) / 2
        shoulder_midpoints.append(shoulder_midpoint)

        left_eye = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_EYE]
        right_eye = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_EYE]
        nose = results.pose_landmarks.landmark[mp_pose.PoseLandmark.NOSE]

        # Calculate head turn angle
        if left_eye and right_eye and nose:
            eye_line_vector = np.array([right_eye.x - left_eye.x, right_eye.y - left_eye.y])
            nose_vector = np.array([nose.x - (left_eye.x + right_eye.x) / 2, nose.y - (left_eye.y + right_eye.y) / 2])
            cosine_angle = np.dot(eye_line_vector, nose_vector) / (np.linalg.norm(eye_line_vector) * np.linalg.norm(nose_vector))
            head_turn_angle = np.arccos(cosine_angle) * (180 / np.pi)
            head_turn_angles.append(head_turn_angle)
        else:
            head_turn_angles.append(None)
    else:
        shoulder_midpoints.append(None)
        head_turn_angles.append(None)

    # Calculate engagement level
    engagement = calculate_engagement(left_hand, right_hand, shoulder_midpoints, head_turn_angles)
    engagement_level.append(engagement)

    # Filter out None values for graphs
    filtered_left_hand = [point for point in left_hand if point is not None]
    filtered_right_hand = [point for point in right_hand if point is not None]
    filtered_shoulder_midpoints = [point for point in shoulder_midpoints if point is not None]
    filtered_head_turn_angles = [point for point in head_turn_angles if point is not None]
    filtered_engagement_level = [e for e in engagement_level if e is not None]

    # Calculate min and max values for y-axis limits
    min_left_hand = min(filtered_left_hand) if filtered_left_hand else 0
    max_left_hand = max(filtered_left_hand) if filtered_left_hand else 1
    min_right_hand = min(filtered_right_hand) if filtered_right_hand else 0
    max_right_hand = max(filtered_right_hand) if filtered_right_hand else 1
    min_shoulder_midpoint = min(filtered_shoulder_midpoints) if filtered_shoulder_midpoints else 0
    max_shoulder_midpoint = max(filtered_shoulder_midpoints) if filtered_shoulder_midpoints else 1
    min_head_turn_angle = min(filtered_head_turn_angles) if filtered_head_turn_angles else 0
    max_head_turn_angle = max(filtered_head_turn_angles) if filtered_head_turn_angles else 1
    min_engagement = 1
    max_engagement = 3

    # Create separate figures for each graph
    plt.figure(figsize=(output_width / 100, 4 * graph_height / 100))  # Adjust graph size

    # Hand Movement Graph
    plt.subplot(4, 1, 1)
    if len(filtered_left_hand) > 0 and len(filtered_right_hand) > 0:
        plt.plot(np.arange(len(filtered_left_hand)), filtered_left_hand, color='red', linewidth=line_width, marker='o', markersize=1, label='Left hand movement')
        plt.plot(np.arange(len(filtered_right_hand)), filtered_right_hand, color='blue', linewidth=line_width, marker='o', markersize=1, label='Right hand movement')
    plt.ylim([min(min_left_hand, min_right_hand), max(max_left_hand, max_right_hand)])
    plt.xlim([0, frame_index])
    plt.axis('on')
    plt.ylabel('Hand Movements', fontsize=12)
    plt.legend()

    # Shoulder Midpoints Graph
    plt.subplot(4, 1, 2)
    if len(filtered_shoulder_midpoints) > 0:
        plt.plot(np.arange(len(filtered_shoulder_midpoints)), filtered_shoulder_midpoints, color='orange', linewidth=line_width, marker='o', markersize=1, label='Shoulder Midpoints')
    plt.ylim([min_shoulder_midpoint, max_shoulder_midpoint])
    plt.xlim([0, frame_index])
    plt.axis('on')
    plt.ylabel('Shoulder Midpoints', fontsize=12)
    plt.legend()

    # Head Turn Angles Graph
    plt.subplot(4, 1, 3)
    if len(filtered_head_turn_angles) > 0:
        plt.plot(np.arange(len(filtered_head_turn_angles)), filtered_head_turn_angles, color='purple', linewidth=line_width, marker='o', markersize=1, label='Head Turn Angles')
    plt.ylim([min_head_turn_angle, max_head_turn_angle])
    plt.xlim([0, frame_index])
    plt.axis('on')
    plt.ylabel('Head Turn Angles', fontsize=12)
    plt.legend()

    # Engagement Level Graph
    engagement_colors = {'attentive': 'green', 'interactive': 'blue', 'inspired': 'red'}
    engagement_numeric = [1 if e == 'attentive' else 2 if e == 'interactive' else 3 for e in filtered_engagement_level]
    plt.subplot(4, 1, 4)
    if len(filtered_engagement_level) > 0:
        plt.plot(np.arange(len(filtered_engagement_level)), engagement_numeric, color='green', linewidth=line_width, marker='o', markersize=1, label='Engagement Level')
        for i, level in enumerate(filtered_engagement_level):
            plt.plot(i, engagement_numeric[i], color=engagement_colors[level], marker='o')
    plt.ylim([0.5, 3.5])
    plt.yticks([1, 2, 3], ['Attentive', 'Interactive', 'Inspired'])
    plt.xlim([0, frame_index])
    plt.axis('on')
    plt.ylabel('Engagement Level', fontsize=12)
    plt.legend()

    plt.tight_layout()

    # Save the graph with labels
    plt.savefig('temp_graph.png', bbox_inches='tight', pad_inches=0, transparent=True)
    plt.clf()

    graph = cv2.imread('temp_graph.png')
    graph = cv2.resize(graph, (output_width, 4 * graph_height))

    return frame, graph

def main():
    global frame_index

    with ThreadPoolExecutor(max_workers=2) as executor:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            future = executor.submit(process_frame, frame, frame_index)
            processed_frame, graph = future.result()

            # Update the frame index
            frame_index += 1

            # Display the input video
            cv2.imshow('Input Video', processed_frame)

            # Display the graph with labels
            cv2.imshow('Graph', graph)

            # Write the frame with graph to the output video
            out.write(cv2.resize(processed_frame, (output_width, output_height)))

            # Break the loop if the 'q' key is pressed
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

            # Slow down the frame rate
            time.sleep(0.1)

    # Release video objects and close OpenCV windows
    cap.release()
    out.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


In [ ]:
# above is fine , some more refinement

In [ ]:
import cv2
import numpy as np
import mediapipe as mp
import matplotlib.pyplot as plt
from concurrent.futures import ThreadPoolExecutor
import time

# Initialize MediaPipe Pose and Holistic
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)
mp_holistic = mp.solutions.holistic
holistic_model = mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

# Load Haar Cascade for face detection
face_haar_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Initialize lists to store data for graphs
left_hand = []
right_hand = []
shoulder_midpoints = []
head_turn_angles = []
engagement_level = []

# Set the graph display parameters
graph_height = 200
line_width = 1

# Set video dimensions
output_width = 800
output_height = 600

# Create a video capture object
cap = cv2.VideoCapture(0)

# Create an output video writer
output_video = 'output_video.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video, fourcc, 20, (output_width, output_height))

# Initialize frame index
frame_index = 0

def calculate_engagement(left_hand, right_hand, shoulder_midpoints, head_turn_angles):
    # Use recent history to determine engagement level
    history_len = 10  # Number of frames to consider for engagement calculation

    # Slicing the recent history
    recent_left_hand = left_hand[-history_len:]
    recent_right_hand = right_hand[-history_len:]
    recent_shoulder_midpoints = shoulder_midpoints[-history_len:]
    recent_head_turn_angles = head_turn_angles[-history_len:]

    # Calculate average movements
    avg_left_hand = np.nanmean([x for x in recent_left_hand if x is not None])
    avg_right_hand = np.nanmean([x for x in recent_right_hand if x is not None])
    avg_shoulder = np.nanmean([x for x in recent_shoulder_midpoints if x is not None])
    avg_head_turn = np.nanmean([x for x in recent_head_turn_angles if x is not None])

    # Engagement calculation based on movement thresholds
    hand_threshold = 0.1  # Movement threshold for hands
    shoulder_threshold = 0.1  # Increased movement threshold for shoulders
    head_turn_threshold = 10.0  # Increased angle threshold for head turns (degrees)

    hand_movement = (avg_left_hand is not None and avg_left_hand > hand_threshold) or (avg_right_hand is not None and avg_right_hand > hand_threshold)
    shoulder_movement = avg_shoulder is not None and avg_shoulder > shoulder_threshold
    head_movement = avg_head_turn is not None and avg_head_turn > head_turn_threshold

    if hand_movement and shoulder_movement and head_movement:
        return 'inspired'
    elif hand_movement and (shoulder_movement or head_movement):
        return 'interactive'
    else:
        return 'attentive'

def process_frame(frame, frame_index):
    global left_hand, right_hand, shoulder_midpoints, head_turn_angles, engagement_level

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame_grey = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_haar_cascade.detectMultiScale(frame_grey)

    # Detect pose in the frame
    results = pose.process(frame_rgb)
    frame_rgb.flags.writeable = False
    hand_results = holistic_model.process(frame_rgb)
    frame_rgb.flags.writeable = True

    # Draw landmarks and calculate distances for left and right hands
    left_hand_sum_distance = 0
    right_hand_sum_distance = 0

    if hand_results.left_hand_landmarks:
        mp_drawing.draw_landmarks(frame, hand_results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
        left_hand_landmarks = hand_results.left_hand_landmarks.landmark
        for landmark in left_hand_landmarks:
            x = landmark.x
            y = landmark.y
            z = landmark.z
            distance = np.sqrt(x**2 + y**2 + z**2)
            left_hand_sum_distance += distance
        left_hand.append(left_hand_sum_distance)
    else:
        left_hand.append(None)

    if hand_results.right_hand_landmarks:
        mp_drawing.draw_landmarks(frame, hand_results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
        right_hand_landmarks = hand_results.right_hand_landmarks.landmark
        for landmark in right_hand_landmarks:
            x = landmark.x
            y = landmark.y
            z = landmark.z
            distance = np.sqrt(x**2 + y**2 + z**2)
            right_hand_sum_distance += distance
        right_hand.append(right_hand_sum_distance)
    else:
        right_hand.append(None)

    # Draw pose landmarks
    if results.pose_landmarks:
        mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        left_shoulder = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER]
        right_shoulder = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER]
        shoulder_midpoint = (left_shoulder.x + right_shoulder.x) / 2
        shoulder_midpoints.append(shoulder_midpoint)

        left_eye = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_EYE]
        right_eye = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_EYE]
        nose = results.pose_landmarks.landmark[mp_pose.PoseLandmark.NOSE]

        # Calculate head turn angle
        if left_eye and right_eye and nose:
            eye_line_vector = np.array([right_eye.x - left_eye.x, right_eye.y - left_eye.y])
            nose_vector = np.array([nose.x - (left_eye.x + right_eye.x) / 2, nose.y - (left_eye.y + right_eye.y) / 2])
            cosine_angle = np.dot(eye_line_vector, nose_vector) / (np.linalg.norm(eye_line_vector) * np.linalg.norm(nose_vector))
            head_turn_angle = np.arccos(cosine_angle) * (180 / np.pi)
            head_turn_angles.append(head_turn_angle)
        else:
            head_turn_angles.append(None)
    else:
        shoulder_midpoints.append(None)
        head_turn_angles.append(None)

    # Calculate engagement level
    engagement = calculate_engagement(left_hand, right_hand, shoulder_midpoints, head_turn_angles)
    engagement_level.append(engagement)

    # Filter out None values for graphs
    filtered_left_hand = [point for point in left_hand if point is not None]
    filtered_right_hand = [point for point in right_hand if point is not None]
    filtered_shoulder_midpoints = [point for point in shoulder_midpoints if point is not None]
    filtered_head_turn_angles = [point for point in head_turn_angles if point is not None]
    filtered_engagement_level = [e for e in engagement_level if e is not None]

    # Calculate min and max values for y-axis limits
    min_left_hand = min(filtered_left_hand) if filtered_left_hand else 0
    max_left_hand = max(filtered_left_hand) if filtered_left_hand else 1
    min_right_hand = min(filtered_right_hand) if filtered_right_hand else 0
    max_right_hand = max(filtered_right_hand) if filtered_right_hand else 1
    min_shoulder_midpoint = min(filtered_shoulder_midpoints) if filtered_shoulder_midpoints else 0
    max_shoulder_midpoint = max(filtered_shoulder_midpoints) if filtered_shoulder_midpoints else 1
    min_head_turn_angle = min(filtered_head_turn_angles) if filtered_head_turn_angles else 0
    max_head_turn_angle = max(filtered_head_turn_angles) if filtered_head_turn_angles else 1
    min_engagement = 1
    max_engagement = 3

    # Create separate figures for each graph
    plt.figure(figsize=(output_width / 100, 4 * graph_height / 100))  # Adjust graph size

    # Hand Movement Graph
    plt.subplot(4, 1, 1)
    if len(filtered_left_hand) > 0 and len(filtered_right_hand) > 0:
        plt.plot(np.arange(len(filtered_left_hand)), filtered_left_hand, color='red', linewidth=line_width, marker='o', markersize=1, label='Left hand movement')
        plt.plot(np.arange(len(filtered_right_hand)), filtered_right_hand, color='blue', linewidth=line_width, marker='o', markersize=1, label='Right hand movement')
    plt.ylim([min(min_left_hand, min_right_hand), max(max_left_hand, max_right_hand)])
    plt.xlim([0, frame_index])
    plt.axis('on')
    plt.ylabel('Hand Movements', fontsize=12)
    plt.legend()

    # Shoulder Midpoints Graph
    plt.subplot(4, 1, 2)
    if len(filtered_shoulder_midpoints) > 0:
        plt.plot(np.arange(len(filtered_shoulder_midpoints)), filtered_shoulder_midpoints, color='orange', linewidth=line_width, marker='o', markersize=1, label='Shoulder Midpoints')
    plt.ylim([min_shoulder_midpoint, max_shoulder_midpoint])
    plt.xlim([0, frame_index])
    plt.axis('on')
    plt.ylabel('Shoulder Midpoints', fontsize=12)
    plt.legend()

    # Head Turn Angles Graph
    plt.subplot(4, 1, 3)
    if len(filtered_head_turn_angles) > 0:
        plt.plot(np.arange(len(filtered_head_turn_angles)), filtered_head_turn_angles, color='purple', linewidth=line_width, marker='o', markersize=1, label='Head Turn Angles')
    plt.ylim([min_head_turn_angle, max_head_turn_angle])
    plt.xlim([0, frame_index])
    plt.axis('on')
    plt.ylabel('Head Turn Angles', fontsize=12)
    plt.legend()

    # Engagement Level Graph
    engagement_colors = {'attentive': 'green', 'interactive': 'blue', 'inspired': 'red'}
    engagement_numeric = [1 if e == 'attentive' else 2 if e == 'interactive' else 3 for e in filtered_engagement_level]
    plt.subplot(4, 1, 4)
    if len(filtered_engagement_level) > 0:
        plt.plot(np.arange(len(filtered_engagement_level)), engagement_numeric, color='green', linewidth=line_width, marker='o', markersize=1, label='Engagement Level')
        for i, level in enumerate(filtered_engagement_level):
            plt.plot(i, engagement_numeric[i], color=engagement_colors[level], marker='o')
    plt.ylim([0.5, 3.5])
    plt.yticks([1, 2, 3], ['Attentive', 'Interactive', 'Inspired'])
    plt.xlim([0, frame_index])
    plt.axis('on')
    plt.ylabel('Engagement Level', fontsize=12)
    plt.legend()

    plt.tight_layout()

    # Save the graph with labels
    plt.savefig('temp_graph.png', bbox_inches='tight', pad_inches=0, transparent=True)
    plt.clf()

    graph = cv2.imread('temp_graph.png')
    graph = cv2.resize(graph, (output_width, 4 * graph_height))

    return frame, graph

def main():
    global frame_index

    with ThreadPoolExecutor(max_workers=2) as executor:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            future = executor.submit(process_frame, frame, frame_index)
            processed_frame, graph = future.result()

            # Update the frame index
            frame_index += 1

            # Display the input video
            cv2.imshow('Input Video', processed_frame)

            # Display the graph with labels
            cv2.imshow('Graph', graph)

            # Write the frame with graph to the output video
            out.write(cv2.resize(processed_frame, (output_width, output_height)))

            # Break the loop if the 'q' key is pressed
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

            # Slow down the frame rate
            time.sleep(0.1)

    # Release video objects and close OpenCV windows
    cap.release()
    out.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


In [ ]:
import cv2
import numpy as np
import mediapipe as mp
import matplotlib.pyplot as plt
from concurrent.futures import ThreadPoolExecutor
import time

# Initialize MediaPipe Pose and Holistic
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)
mp_holistic = mp.solutions.holistic
holistic_model = mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

# Load Haar Cascade for face detection
face_haar_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Initialize lists to store data for graphs
left_hand = []
right_hand = []
shoulder_midpoints = []
head_turn_angles = []
engagement_level = []

# Set the graph display parameters
graph_height = 200
line_width = 1

# Set video dimensions
output_width = 800
output_height = 600

# Create a video capture object
cap = cv2.VideoCapture(0)

# Create an output video writer
output_video = 'output_video.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video, fourcc, 20, (output_width, output_height))

# Initialize frame index
frame_index = 0

def calculate_engagement(left_hand, right_hand, shoulder_midpoints, head_turn_angles):
    # Use recent history to determine engagement level
    history_len = 10  # Number of frames to consider for engagement calculation

    # Slicing the recent history
    recent_left_hand = left_hand[-history_len:]
    recent_right_hand = right_hand[-history_len:]
    recent_shoulder_midpoints = shoulder_midpoints[-history_len:]
    recent_head_turn_angles = head_turn_angles[-history_len:]

    # Calculate average movements
    avg_left_hand = np.nanmean([x for x in recent_left_hand if x is not None])
    avg_right_hand = np.nanmean([x for x in recent_right_hand if x is not None])
    avg_shoulder = np.nanmean([x for x in recent_shoulder_midpoints if x is not None])
    avg_head_turn = np.nanmean([x for x in recent_head_turn_angles if x is not None])

    # Engagement calculation based on movement thresholds
    hand_threshold = 0.1  # Movement threshold for hands
    shoulder_threshold = 0.05  # Movement threshold for shoulders
    head_turn_threshold = 5.0  # Angle threshold for head turns (degrees)

    left_hand_movement = avg_left_hand is not None and avg_left_hand > hand_threshold
    right_hand_movement = avg_right_hand is not None and avg_right_hand > hand_threshold
    shoulder_movement = avg_shoulder is not None and avg_shoulder > shoulder_threshold
    head_movement = avg_head_turn is not None and avg_head_turn > head_turn_threshold

    if left_hand_movement and right_hand_movement:
        return 'inspired'
    elif left_hand_movement or right_hand_movement:
        return 'interactive'
    else:
        return 'attentive'

def process_frame(frame, frame_index):
    global left_hand, right_hand, shoulder_midpoints, head_turn_angles, engagement_level

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame_grey = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_haar_cascade.detectMultiScale(frame_grey)

    # Detect pose in the frame
    results = pose.process(frame_rgb)
    frame_rgb.flags.writeable = False
    hand_results = holistic_model.process(frame_rgb)
    frame_rgb.flags.writeable = True

    # Draw landmarks and calculate distances for left and right hands
    left_hand_sum_distance = 0
    right_hand_sum_distance = 0

    if hand_results.left_hand_landmarks:
        mp_drawing.draw_landmarks(frame, hand_results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
        left_hand_landmarks = hand_results.left_hand_landmarks.landmark
        for landmark in left_hand_landmarks:
            x = landmark.x
            y = landmark.y
            z = landmark.z
            distance = np.sqrt(x**2 + y**2 + z**2)
            left_hand_sum_distance += distance
        left_hand.append(left_hand_sum_distance)
    else:
        left_hand.append(None)

    if hand_results.right_hand_landmarks:
        mp_drawing.draw_landmarks(frame, hand_results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
        right_hand_landmarks = hand_results.right_hand_landmarks.landmark
        for landmark in right_hand_landmarks:
            x = landmark.x
            y = landmark.y
            z = landmark.z
            distance = np.sqrt(x**2 + y**2 + z**2)
            right_hand_sum_distance += distance
        right_hand.append(right_hand_sum_distance)
    else:
        right_hand.append(None)

    # Draw pose landmarks
    if results.pose_landmarks:
        mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        left_shoulder = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER]
        right_shoulder = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER]
        shoulder_midpoint = (left_shoulder.x + right_shoulder.x) / 2
        shoulder_midpoints.append(shoulder_midpoint)

        left_eye = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_EYE]
        right_eye = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_EYE]
        nose = results.pose_landmarks.landmark[mp_pose.PoseLandmark.NOSE]

        # Calculate head turn angle
        if left_eye and right_eye and nose:
            eye_line_vector = np.array([right_eye.x - left_eye.x, right_eye.y - left_eye.y])
            nose_vector = np.array([nose.x - (left_eye.x + right_eye.x) / 2, nose.y - (left_eye.y + right_eye.y) / 2])
            cosine_angle = np.dot(eye_line_vector, nose_vector) / (np.linalg.norm(eye_line_vector) * np.linalg.norm(nose_vector))
            head_turn_angle = np.arccos(cosine_angle) * (180 / np.pi)
            head_turn_angles.append(head_turn_angle)
        else:
            head_turn_angles.append(None)
    else:
        shoulder_midpoints.append(None)
        head_turn_angles.append(None)

    # Calculate engagement level
    engagement = calculate_engagement(left_hand, right_hand, shoulder_midpoints, head_turn_angles)
    engagement_level.append(engagement)

    # Filter out None values for graphs
    filtered_left_hand = [point for point in left_hand if point is not None]
    filtered_right_hand = [point for point in right_hand if point is not None]
    filtered_shoulder_midpoints = [point for point in shoulder_midpoints if point is not None]
    filtered_head_turn_angles = [point for point in head_turn_angles if point is not None]
    filtered_engagement_level = [e for e in engagement_level if e is not None]

    # Calculate min and max values for y-axis limits
    min_left_hand = min(filtered_left_hand) if filtered_left_hand else 0
    max_left_hand = max(filtered_left_hand) if filtered_left_hand else 1
    min_right_hand = min(filtered_right_hand) if filtered_right_hand else 0
    max_right_hand = max(filtered_right_hand) if filtered_right_hand else 1
    min_shoulder_midpoint = min(filtered_shoulder_midpoints) if filtered_shoulder_midpoints else 0
    max_shoulder_midpoint = max(filtered_shoulder_midpoints) if filtered_shoulder_midpoints else 1
    min_head_turn_angle = min(filtered_head_turn_angles) if filtered_head_turn_angles else 0
    max_head_turn_angle = max(filtered_head_turn_angles) if filtered_head_turn_angles else 1
    min_engagement = 1
    max_engagement = 3

    # Create separate figures for each graph
    plt.figure(figsize=(output_width / 100, 4 * graph_height / 100))  # Adjust graph size

    # Hand Movement Graph
    plt.subplot(4, 1, 1)
    if len(filtered_left_hand) > 0 and len(filtered_right_hand) > 0:
        plt.plot(np.arange(len(filtered_left_hand)), filtered_left_hand, color='red', linewidth=line_width, marker='o', markersize=1, label='Left hand movement')
        plt.plot(np.arange(len(filtered_right_hand)), filtered_right_hand, color='blue', linewidth=line_width, marker='o', markersize=1, label='Right hand movement')
    plt.ylim([min(min_left_hand, min_right_hand), max(max_left_hand, max_right_hand)])
    plt.xlim([0, frame_index])
    plt.axis('on')
    plt.ylabel('Hand Movements', fontsize=12)
    plt.legend()

    # Shoulder Midpoints Graph
    plt.subplot(4, 1, 2)
    if len(filtered_shoulder_midpoints) > 0:
        plt.plot(np.arange(len(filtered_shoulder_midpoints)), filtered_shoulder_midpoints, color='orange', linewidth=line_width, marker='o', markersize=1, label='Shoulder Midpoints')
    plt.ylim([min_shoulder_midpoint, max_shoulder_midpoint])
    plt.xlim([0, frame_index])
    plt.axis('on')
    plt.ylabel('Shoulder Midpoints', fontsize=12)
    plt.legend()

    # Head Turn Angles Graph
    plt.subplot(4, 1, 3)
    if len(filtered_head_turn_angles) > 0:
        plt.plot(np.arange(len(filtered_head_turn_angles)), filtered_head_turn_angles, color='purple', linewidth=line_width, marker='o', markersize=1, label='Head Turn Angles')
    plt.ylim([min_head_turn_angle, max_head_turn_angle])
    plt.xlim([0, frame_index])
    plt.axis('on')
    plt.ylabel('Head Turn Angles', fontsize=12)
    plt.legend()

    # Engagement Level Graph
    engagement_colors = {'attentive': 'green', 'interactive': 'blue', 'inspired': 'red'}
    engagement_numeric = [1 if e == 'attentive' else 2 if e == 'interactive' else 3 for e in filtered_engagement_level]
    plt.subplot(4, 1, 4)
    if len(filtered_engagement_level) > 0:
        plt.plot(np.arange(len(filtered_engagement_level)), engagement_numeric, color='green', linewidth=line_width, marker='o', markersize=1, label='Engagement Level')
        for i, level in enumerate(filtered_engagement_level):
            plt.plot(i, engagement_numeric[i], color=engagement_colors[level], marker='o')
    plt.ylim([0.5, 3.5])
    plt.yticks([1, 2, 3], ['Attentive', 'Interactive', 'Inspired'])
    plt.xlim([0, frame_index])
    plt.axis('on')
    plt.ylabel('Engagement Level', fontsize=12)
    plt.legend()

    plt.tight_layout()

    # Save the graph with labels
    plt.savefig('temp_graph.png', bbox_inches='tight', pad_inches=0, transparent=True)
    plt.clf()

    graph = cv2.imread('temp_graph.png')
    graph = cv2.resize(graph, (output_width, 4 * graph_height))

    return frame, graph

def main():
    global frame_index

    with ThreadPoolExecutor(max_workers=2) as executor:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            future = executor.submit(process_frame, frame, frame_index)
            processed_frame, graph = future.result()

            # Update the frame index
            frame_index += 1

            # Display the input video
            cv2.imshow('Input Video', processed_frame)

            # Display the graph with labels
            cv2.imshow('Graph', graph)

            # Write the frame with graph to the output video
            out.write(cv2.resize(processed_frame, (output_width, output_height)))

            # Break the loop if the 'q' key is pressed
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

            # Slow down the frame rate
            time.sleep(0.1)

    # Release video objects and close OpenCV windows
    cap.release()
    out.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


In [ ]:
# below code is with streamlit

In [1]:
import streamlit as st
import cv2
import numpy as np
import mediapipe as mp
from concurrent.futures import ThreadPoolExecutor
import time
import matplotlib.pyplot as plt

# Initialize MediaPipe Pose and Holistic
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)
mp_holistic = mp.solutions.holistic
holistic_model = mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

# Load Haar Cascade for face detection
face_haar_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Initialize lists to store data for graphs
left_hand = []
right_hand = []
shoulder_midpoints = []
head_turn_angles = []
engagement_level = []

# Set the graph display parameters
graph_height = 200
line_width = 1

# Set video dimensions
output_width = 800
output_height = 600

# Streamlit interface
st.title("Real-time Pose and Engagement Analysis")
st.sidebar.title("Controls")

# Camera toggle
camera_enabled = st.sidebar.checkbox("Enable Camera", value=True)

# Create a video capture object
cap = None
if camera_enabled:
    cap = cv2.VideoCapture(0)

def calculate_engagement(left_hand, right_hand, shoulder_midpoints, head_turn_angles):
    # Use recent history to determine engagement level
    history_len = 10  # Number of frames to consider for engagement calculation

    # Slicing the recent history
    recent_left_hand = left_hand[-history_len:]
    recent_right_hand = right_hand[-history_len:]
    recent_shoulder_midpoints = shoulder_midpoints[-history_len:]
    recent_head_turn_angles = head_turn_angles[-history_len:]

    # Calculate average movements
    avg_left_hand = np.nanmean([x for x in recent_left_hand if x is not None])
    avg_right_hand = np.nanmean([x for x in recent_right_hand if x is not None])
    avg_shoulder = np.nanmean([x for x in recent_shoulder_midpoints if x is not None])
    avg_head_turn = np.nanmean([x for x in recent_head_turn_angles if x is not None])

    # Engagement calculation based on movement thresholds
    hand_threshold = 0.1  # Movement threshold for hands
    shoulder_threshold = 0.05  # Movement threshold for shoulders
    head_turn_threshold = 5.0  # Angle threshold for head turns (degrees)

    left_hand_movement = avg_left_hand is not None and avg_left_hand > hand_threshold
    right_hand_movement = avg_right_hand is not None and avg_right_hand > hand_threshold
    shoulder_movement = avg_shoulder is not None and avg_shoulder > shoulder_threshold
    head_movement = avg_head_turn is not None and avg_head_turn > head_turn_threshold

    if left_hand_movement and right_hand_movement:
        return 'inspired'
    elif left_hand_movement or right_hand_movement:
        return 'interactive'
    else:
        return 'attentive'

def process_frame(frame, frame_index):
    global left_hand, right_hand, shoulder_midpoints, head_turn_angles, engagement_level

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame_grey = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_haar_cascade.detectMultiScale(frame_grey)

    # Detect pose in the frame
    results = pose.process(frame_rgb)
    frame_rgb.flags.writeable = False
    hand_results = holistic_model.process(frame_rgb)
    frame_rgb.flags.writeable = True

    # Draw landmarks and calculate distances for left and right hands
    left_hand_sum_distance = 0
    right_hand_sum_distance = 0

    if hand_results.left_hand_landmarks:
        mp_drawing.draw_landmarks(frame, hand_results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
        left_hand_landmarks = hand_results.left_hand_landmarks.landmark
        for landmark in left_hand_landmarks:
            x = landmark.x
            y = landmark.y
            z = landmark.z
            distance = np.sqrt(x**2 + y**2 + z**2)
            left_hand_sum_distance += distance
        left_hand.append(left_hand_sum_distance)
    else:
        left_hand.append(None)

    if hand_results.right_hand_landmarks:
        mp_drawing.draw_landmarks(frame, hand_results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
        right_hand_landmarks = hand_results.right_hand_landmarks.landmark
        for landmark in right_hand_landmarks:
            x = landmark.x
            y = landmark.y
            z = landmark.z
            distance = np.sqrt(x**2 + y**2 + z**2)
            right_hand_sum_distance += distance
        right_hand.append(right_hand_sum_distance)
    else:
        right_hand.append(None)

    # Draw pose landmarks
    if results.pose_landmarks:
        mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        left_shoulder = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER]
        right_shoulder = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER]
        shoulder_midpoint = (left_shoulder.x + right_shoulder.x) / 2
        shoulder_midpoints.append(shoulder_midpoint)

        left_eye = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_EYE]
        right_eye = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_EYE]
        nose = results.pose_landmarks.landmark[mp_pose.PoseLandmark.NOSE]

        # Calculate head turn angle
        if left_eye and right_eye and nose:
            eye_line_vector = np.array([right_eye.x - left_eye.x, right_eye.y - left_eye.y])
            nose_vector = np.array([nose.x - (left_eye.x + right_eye.x) / 2, nose.y - (left_eye.y + right_eye.y) / 2])
            cosine_angle = np.dot(eye_line_vector, nose_vector) / (np.linalg.norm(eye_line_vector) * np.linalg.norm(nose_vector))
            head_turn_angle = np.arccos(cosine_angle) * (180 / np.pi)
            head_turn_angles.append(head_turn_angle)
        else:
            head_turn_angles.append(None)
    else:
        shoulder_midpoints.append(None)
        head_turn_angles.append(None)

    # Calculate engagement level
    engagement = calculate_engagement(left_hand, right_hand, shoulder_midpoints, head_turn_angles)
    engagement_level.append(engagement)

    # Filter out None values for graphs
    filtered_left_hand = [point for point in left_hand if point is not None]
    filtered_right_hand = [point for point in right_hand if point is not None]
    filtered_shoulder_midpoints = [point for point in shoulder_midpoints if point is not None]
    filtered_head_turn_angles = [point for point in head_turn_angles if point is not None]
    filtered_engagement_level = [e for e in engagement_level if e is not None]

    # Create separate figures for each graph
    fig, axes = plt.subplots(4, 1, figsize=(output_width / 100, 4 * graph_height / 100))  # Adjust graph size

    # Hand Movement Graph
    axes[0].plot(filtered_left_hand, color='red', linewidth=line_width, marker='o', markersize=1, label='Left hand movement')
    axes[0].plot(filtered_right_hand, color='blue', linewidth=line_width, marker='o', markersize=1, label='Right hand movement')
    axes[0].set_ylim([min(filtered_left_hand, default=0), max(filtered_left_hand, default=1)])
    axes[0].set_ylabel('Hand Movements')
    axes[0].legend()

    # Shoulder Midpoints Graph
    axes[1].plot(filtered_shoulder_midpoints, color='orange', linewidth=line_width, marker='o', markersize=1, label='Shoulder Midpoints')
    axes[1].set_ylim([min(filtered_shoulder_midpoints, default=0), max(filtered_shoulder_midpoints, default=1)])
    axes[1].set_ylabel('Shoulder Midpoints')
    axes[1].legend()

    # Head Turn Angles Graph
    axes[2].plot(filtered_head_turn_angles, color='purple', linewidth=line_width, marker='o', markersize=1, label='Head Turn Angles')
    axes[2].set_ylim([min(filtered_head_turn_angles, default=0), max(filtered_head_turn_angles, default=1)])
    axes[2].set_ylabel('Head Turn Angles')
    axes[2].legend()

    # Engagement Level Graph
    engagement_colors = {'attentive': 'green', 'interactive': 'blue', 'inspired': 'red'}
    engagement_numeric = [1 if e == 'attentive' else 2 if e == 'interactive' else 3 for e in filtered_engagement_level]
    axes[3].plot(engagement_numeric, color='green', linewidth=line_width, marker='o', markersize=1, label='Engagement Level')
    for i, level in enumerate(filtered_engagement_level):
        axes[3].plot(i, engagement_numeric[i], color=engagement_colors[level], marker='o')
    axes[3].set_ylim([0.5, 3.5])
    axes[3].set_yticks([1, 2, 3])
    axes[3].set_yticklabels(['Attentive', 'Interactive', 'Inspired'])
    axes[3].set_ylabel('Engagement Level')
    axes[3].legend()

    plt.tight_layout()

    return frame, fig

# Main loop
def main():
    global frame_index

    frame_index = 0

    if camera_enabled:
        stframe = st.empty()
        graphframe = st.empty()
        with ThreadPoolExecutor(max_workers=2) as executor:
            while cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                    break

                future = executor.submit(process_frame, frame, frame_index)
                processed_frame, fig = future.result()

                frame_index += 1

                # Display the frame
                stframe.image(cv2.cvtColor(processed_frame, cv2.COLOR_BGR2RGB))

                # Display the graph
                graphframe.pyplot(fig)

                time.sleep(0.1)

        cap.release()

if __name__ == "__main__":
    main()


2024-06-24 11:26:21.280079: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-24 11:26:21.285223: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-24 11:26:21.349391: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-24 11:26:22.610230: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
I0000 00:00:1719208584.650944    7031 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1719208584.656879    8104 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 Mesa 24.0.5-1ubuntu1), renderer: AMD Radeon Graphics (radeonsi, renoir, LLVM 17